In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [3]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation, apply_checking_circuit
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.5.1', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

simulation_backend = "qasm_simulator"

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibm_auckland')
device

<IBMQBackend('ibm_auckland') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
device.configuration().to_dict()['max_shots']

100000

In [9]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [10]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [11]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_front.qasm")

In [12]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_mid.qasm")

In [13]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_end.qasm")

In [14]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30535/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [15]:
orign_circuit.draw()

┌───┐       ┌────────────────┐┌────────────┐   ┌─────────────────┐»
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘ │ └─────────────────┘»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                           │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────────────────────────■─────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                      »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘                      »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├────────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘                                    »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──────────────────────────────────────»
     └─────────────────┘  └────────────┘                                      »
q_6: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«      ┌──────────────┐                                                     »
«q_0: ─┤ Rz(-0.23807) ├─────────────────────────────────────────────────────»
«     ┌┴──────────────┤┌──────────────┐                                     »
«q_1: ┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├─────────────────────────────────────»
«     └───────────────┘├─────────────┬┘  ┌────────────┐                     »
«q_2: ────────■────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├─────────────────────»
«             │        └─────────────┘ ┌─┴────────────┴─┐  ┌──────────────┐ »
«q_3: ────────■───────────────■────────┤ Ry(2.6997e-05) ├──┤ Rz(0.022658) ├─»
«                             │        └────────────────┘  ├──────────────┤ »
«q_4: ────────────────────────■────────────────■───────────┤ Ry(0.025251) ├─»
«                                              │         ┌─┴──────────────┴┐»
«q_5: ─────────────────────────────────────────■─────────┤ Ry(-1.0324e-05) ├»
«                                                        └─────────────────┘»
«q_6: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«                      
«q_2: ─────────────────
«                      
«q_3: ─────────────────
«       ┌───────────┐  
«q_4: ──┤ Rz(1.011) ├──
«     ┌─┴───────────┴─┐
«q_5: ┤ Rz(-0.046623) ├
«     └───────────────┘
«q_6: ─────────────────
«

In [16]:
#checking circuit

In [17]:
phase_y_front = [2.113295958493177e-07,0.0054976714345856634, 0.020630778702533964, 3.40008274438278e-05, 0.02524649597343495, -4.81317308598186e-05]
phase_z_front = [1.3165147216569606, -0.07815775859934079, 0.006091440853078919, 1.528289688855932, -0.009893684371252858, 1.3655876587592497]
phase_y_end = [-7.269945363221252e-06, 0.0055856209568994186, 0.020669506733621706, 2.6996665334365784e-05, 0.02525084679082589, -1.0323843699871446e-05]
phase_z_end = [-0.23806644958543488, -0.21685946937956296, 1.2696203088392648, 0.02265810653600898, 1.010960593326885, -0.04662312860318999]
x_front = [True, False, False, True, False, False]

In [18]:
qubit_id = 5
circuit = QuantumCircuit(7)
apply_checking_circuit(circuit, [qubit_id], [6], side = 'front', phase_y = phase_y_front[qubit_id], phase_z = phase_z_front[qubit_id], x = x_front[qubit_id])
circuit += orign_circuit
apply_checking_circuit(circuit, [qubit_id], [6], side = 'end', phase_y = phase_y_end[qubit_id], phase_z = phase_z_end[qubit_id])

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30535/3095446317.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += orign_circuit


In [19]:
circuit.draw()

┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──────────────────────»
            └───┘                                                      »
«     ┌─────────────────┐  ┌──────────────┐                                   »
«q_0: ┤ Ry(-7.2699e-06) ├──┤ Rz(-0.23807) ├───────────────────────────────────»
«     └─────────────────┘ ┌┴──────────────┤ ┌──────────────┐                  »
«q_1: ─────────■──────────┤ Ry(0.0055856) ├─┤ Rz(-0.21686) ├──────────────────»
«              │          └───────────────┘ ├─────────────┬┘  ┌────────────┐  »
«q_2: ─────────■──────────────────■─────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├──»
«                                 │         └─────────────┘ ┌─┴────────────┴─┐»
«q_3: ────────────────────────────■────────────────■────────┤ Ry(2.6997e-05) ├»
«                                                  │        └────────────────┘»
«q_4: ─────────────────────────────────────────────■────────────────■─────────»
«      ┌────────────────┐┌─────────────────┐ ┌────────────┐         │         »
«q_5: ─┤ Ry(4.8132e-05) ├┤ Ry(-4.8132e-05) ├─┤ Rz(1.3656) ├─────────■─────────»
«      └────────────────┘└─────────────────┘ └────────────┘                   »
«q_6: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«                                                                              »
«q_0: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q_1: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q_2: ─────────────────────────────────────────────────────────────────────────»
«       ┌──────────────┐                                                       »
«q_3: ──┤ Rz(0.022658) ├───────────────────────────────────────────────────────»
«       ├──────────────┤   ┌───────────┐                                       »
«q_4: ──┤ Ry(0.025251) ├───┤ Rz(1.011) ├───────────────────────────────────────»
«     ┌─┴──────────────┴┐┌─┴───────────┴─┐┌──────────────┐┌────────────────┐   »
«q_5: ┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├┤ Rz(0.046623) ├┤ Ry(1.0324e-05) ├─■─»
«     └─────────────────┘└───────────────┘└──────────────┘└────────────────┘ │ »
«q_6: ───────────────────────────────────────────────────────────────────────■─»
«                                                                              »
«                                         
«q_0: ────────────────────────────────────
«                                         
«q_1: ────────────────────────────────────
«                                         
«q_2: ────────────────────────────────────
«                                         
«q_3: ────────────────────────────────────
«                                         
«q_4: ────────────────────────────────────
«     ┌─────────────────┐┌───────────────┐
«q_5: ┤ Ry(-1.0324e-05) ├┤ Rz

In [20]:
pauli_list = read_from_file('BeH2_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [21]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 6,num_qargs = 7)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YXYYXY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_5: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_6: ────────────────
                     
YXYZXZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_3: ────────────────
      ┌────────────┐ 
q_4: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_5: ────────────────
                     
q_6: ────────────────
                     
ZXZYXY
                     
q_0: ────────────────
      ┌────────────┐ 
q_1: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_2: ────────────────
     ┌──────────────┐
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_5: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_6: ────────────────
           

In [22]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30535/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [23]:
circuit_list[-2].draw()

┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──────────────────────»
            └───┘                                                      »
«     ┌─────────────────┐  ┌──────────────┐ ┌──────────────┐                  »
«q_0: ┤ Ry(-7.2699e-06) ├──┤ Rz(-0.23807) ├─┤ U(π/2,0,π/2) ├──────────────────»
«     └─────────────────┘ ┌┴──────────────┤ ├──────────────┤ ┌──────────────┐ »
«q_1: ─────────■──────────┤ Ry(0.0055856) ├─┤ Rz(-0.21686) ├─┤ U(π/2,0,π/2) ├─»
«              │          └───────────────┘ ├─────────────┬┘ └┬────────────┬┘ »
«q_2: ─────────■──────────────────■─────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├──»
«                                 │         └─────────────┘ ┌─┴────────────┴─┐»
«q_3: ────────────────────────────■────────────────■────────┤ Ry(2.6997e-05) ├»
«                                                  │        └────────────────┘»
«q_4: ─────────────────────────────────────────────■────────────────■─────────»
«      ┌────────────────┐┌─────────────────┐ ┌────────────┐         │         »
«q_5: ─┤ Ry(4.8132e-05) ├┤ Ry(-4.8132e-05) ├─┤ Rz(1.3656) ├─────────■─────────»
«      └────────────────┘└─────────────────┘ └────────────┘                   »
«q_6: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«                                                                              »
«q_0: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q_1: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q_2: ─────────────────────────────────────────────────────────────────────────»
«       ┌──────────────┐  ┌──────────────┐                                     »
«q_3: ──┤ Rz(0.022658) ├──┤ U(π/2,0,π/2) ├─────────────────────────────────────»
«       ├──────────────┤  └┬───────────┬─┘┌──────────────┐                     »
«q_4: ──┤ Ry(0.025251) ├───┤ Rz(1.011) ├──┤ U(π/2,0,π/2) ├─────────────────────»
«     ┌─┴──────────────┴┐┌─┴───────────┴─┐├──────────────┤┌────────────────┐   »
«q_5: ┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├┤ Rz(0.046623) ├┤ Ry(1.0324e-05) ├─■─»
«     └─────────────────┘└───────────────┘└──────────────┘└────────────────┘ │ »
«q_6: ───────────────────────────────────────────────────────────────────────■─»
«                                                                              »
«                                         
«q_0: ────────────────────────────────────
«                                         
«q_1: ────────────────────────────────────
«                                         
«q_2: ────────────────────────────────────
«                                         
«q_3: ────────────────────────────────────
«                                         
«q_4: ────────────────────────────────────
«     ┌─────────────────┐┌───────────────┐
«q_5: ┤ Ry(-1.0324e-05) ├┤ Rz

In [24]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [25]:
cuts = [(circuit.qubits[5], 5),(circuit.qubits[5], 10)]

In [26]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

            ┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──

            ┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──

            ┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──

            ┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──

            ┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──

...

            ┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──

            ┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──

            ┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──

            ┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──

            ┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──

In [27]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(7, 'q'), 0): ((0, Qubit(QuantumRegister(6, 'q'), 5)),), Qubit(QuantumRegister(7, 'q'), 1): ((0, Qubit(QuantumRegister(6, 'q'), 1)),), Qubit(QuantumRegister(7, 'q'), 2): ((0, Qubit(QuantumRegister(6, 'q'), 2)),), Qubit(QuantumRegister(7, 'q'), 3): ((0, Qubit(QuantumRegister(6, 'q'), 0)),), Qubit(QuantumRegister(7, 'q'), 4): ((0, Qubit(QuantumRegister(6, 'q'), 4)),), Qubit(QuantumRegister(7, 'q'), 5): ((1, Qubit(QuantumRegister(3, 'q'), 2)), (0, Qubit(QuantumRegister(6, 'q'), 3)), (1, Qubit(QuantumRegister(3, 'q'), 0))), Qubit(QuantumRegister(7, 'q'), 6): ((1, Qubit(QuantumRegister(3, 'q'), 1)),)}


In [28]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [29]:
total_variants

24

In [30]:
shots = 10000 * total_variants

In [31]:
fragments_list[0][0].draw()

┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘                      »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └─┬──────────────┬┘┌─┴────────────┴─┐               │                    »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
       └────┬───┬─────┘ ├────────────────┤┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └────────────────┘└────────────┘   └─────────────────┘»
«                                                         ┌────────────────┐»
«q_0: ────────────────────────■─────────────────────────■─┤ Ry(2.6997e-05) ├»
«     ┌───────────────┐       │        ┌──────────────┐ │ └─┬────────────┬─┘»
«q_1: ┤ Ry(0.0055856) ├───────┼────────┤ Rz(-0.21686) ├─┼───┤ U(π/2,0,π) ├──»
«     └───────────────┘       │        ├─────────────┬┘ │   ├────────────┤  »
«q_2: ────────────────────────■────────┤ Ry(0.02067) ├──┼───┤ Rz(1.2696) ├──»
«                                      └─────────────┘  │   └────────────┘  »
«q_3: ──────────────────────────────────────────────────┼─────────■─────────»
«                                                       │         │         »
«q_4: ──────────────────────────────────────────────────■─────────■─────────»
«      ┌──────────────┐┌──────────────┐                                     »
«q_5: ─┤ Rz(-0.23807) ├┤ U(π/2,0,π/2) ├─────────────────────────────────────»
«      └──────────────┘└──────────────┘                                     »
«       ┌──────────────┐  ┌──────────────┐              
«q_0: ──┤ Rz(0.022658) ├──┤ U(π/2,0,π/2) ├──────────────
«       └──────────────┘  └──────────────┘              
«q_1: ──────────────────────────────────────────────────
«       ┌──────────────┐                                
«q_2: ──┤ U(π/2,0,π/2) ├────────────────────────────────
«     ┌─┴──────────────┴┐┌───────────────┐              
«q_3: ┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├──────────────
«     └─┬──────────────┬┘└─┬───────────┬─┘┌────────────┐
«q_4: ──┤ Ry(0.025251) ├───┤ Rz(1.011) ├──┤ U(π/2,0,π) ├
«       └──────────────┘   └───────────┘  └────────────┘
«q_5: ──────────────────────────────────────────────────
«

In [32]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘                      »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └─┬──────────────┬┘┌─┴────────────┴─┐               │                    »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
       └────┬───┬─────┘ ├────────────────┤┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └───

            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘                      »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └─┬──────────────┬┘┌─┴────────────┴─┐               │                    »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
       └────┬───┬─────┘ ├────────────────┤┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └───────────────

            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘                      »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └─┬──────────────┬┘┌─┴────────────┴─┐               │                    »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
       └────┬───┬─────┘ ├────────────────┤┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └───────────────

            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘                      »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └─┬──────────────┬┘┌─┴────────────┴─┐               │                    »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
       └────┬───┬─────┘ ├────────────────┤┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └───────────────

            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘                      »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └─┬──────────────┬┘┌─┴────────────┴─┐               │                    »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
       └────┬───┬─────┘ ├────────────────┤┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └───────────────

            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘                      »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └─┬──────────────┬┘┌─┴────────────┴─┐               │                    »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
       └────┬───┬─────┘ ├────────────────┤┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └───────────────

In [33]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [34]:
wire_path_map

{Qubit(QuantumRegister(7, 'q'), 0): ((0, Qubit(QuantumRegister(6, 'q'), 5)),),
 Qubit(QuantumRegister(7, 'q'), 1): ((0, Qubit(QuantumRegister(6, 'q'), 1)),),
 Qubit(QuantumRegister(7, 'q'), 2): ((0, Qubit(QuantumRegister(6, 'q'), 2)),),
 Qubit(QuantumRegister(7, 'q'), 3): ((0, Qubit(QuantumRegister(6, 'q'), 0)),),
 Qubit(QuantumRegister(7, 'q'), 4): ((0, Qubit(QuantumRegister(6, 'q'), 4)),),
 Qubit(QuantumRegister(7, 'q'), 5): ((1, Qubit(QuantumRegister(3, 'q'), 2)),
  (0, Qubit(QuantumRegister(6, 'q'), 3)),
  (1, Qubit(QuantumRegister(3, 'q'), 0))),
 Qubit(QuantumRegister(7, 'q'), 6): ((1, Qubit(QuantumRegister(3, 'q'), 1)),)}

In [35]:
num_fragments = len(fragments)
num_fragments

2

In [36]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

34

In [37]:
fragments[0].draw()

┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘                      »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └─┬──────────────┬┘┌─┴────────────┴─┐               │                    »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
       └────┬───┬─────┘ ├────────────────┤┌────────────┐ │ ┌─────────────────┐»
q_5: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            └───┘       └────────────────┘└────────────┘   └─────────────────┘»
«                                            ┌────────────────┐»
«q_0: ──────────────────■──────────────────■─┤ Ry(2.6997e-05) ├»
«     ┌───────────────┐ │ ┌──────────────┐ │ └┬──────────────┬┘»
«q_1: ┤ Ry(0.0055856) ├─┼─┤ Rz(-0.21686) ├─┼──┤ U(π/2,0,π/2) ├─»
«     └───────────────┘ │ ├─────────────┬┘ │  └┬────────────┬┘ »
«q_2: ──────────────────■─┤ Ry(0.02067) ├──┼───┤ Rz(1.2696) ├──»
«                         └─────────────┘  │   └────────────┘  »
«q_3: ─────────────────────────────────────┼─────────■─────────»
«                                          │         │         »
«q_4: ─────────────────────────────────────■─────────■─────────»
«      ┌──────────────┐                                        »
«q_5: ─┤ Rz(-0.23807) ├────────────────────────────────────────»
«      └──────────────┘                                        »
«       ┌──────────────┐                                  
«q_0: ──┤ Rz(0.022658) ├──────────────────────────────────
«       └──────────────┘                                  
«q_1: ────────────────────────────────────────────────────
«       ┌──────────────┐                                  
«q_2: ──┤ U(π/2,0,π/2) ├──────────────────────────────────
«     ┌─┴──────────────┴┐┌───────────────┐                
«q_3: ┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├────────────────
«     └─┬──────────────┬┘└─┬───────────┬─┘┌──────────────┐
«q_4: ──┤ Ry(0.025251) ├───┤ Rz(1.011) ├──┤ U(π/2,0,π/2) ├
«       └──────────────┘   └───────────┘  └──────────────┘
«q_5: ────────────────────────────────────────────────────
«

In [38]:
fragments[1].draw()

┌──────────────┐ ┌────────────────┐                  ┌─────────────────┐»
q_0: ──┤ Rz(0.046623) ├─┤ Ry(1.0324e-05) ├──────────■───────┤ Ry(-1.0324e-05) ├»
       └────┬───┬─────┘ └────────────────┘          │       └──────┬───┬──────┘»
q_1: ───────┤ H ├──────────────────────────■────────■──────────────┤ H ├───────»
     ┌──────┴───┴──────┐  ┌────────────┐   │ ┌─────────────┐ ┌─────┴───┴──────┐»
q_2: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├───■─┤ Rz(-1.3656) ├─┤ Ry(4.8132e-05) ├»
     └─────────────────┘  └────────────┘     └─────────────┘ └────────────────┘»
«     ┌───────────────┐┌──────────────┐
«q_0: ┤ Rz(-0.046623) ├┤ U(π/2,0,π/2) ├
«     └───────────────┘└──────────────┘
«q_1: ─────────────────────────────────
«                                      
«q_2: ─────────────────────────────────
«

In [39]:
hardware_index = 0

In [40]:
print(shots)

240000


In [41]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [42]:
#contains 34 basis
len(tomography_circuits)

34

In [43]:
#contains two fragments
len(tomography_circuits[0])

2

In [44]:
len(tomography_circuits[0][1])

1

In [45]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [46]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [49]:
#[12,13,14,11,8,9]
#[11,13,14,9,8,12]

In [48]:
multi_exp_circuits[hardware_index][-200].draw()

░        ┌───┐       ┌────────────────┐»
  q_0: ────────────────────░────────┤ X ├───────┤ Ry(3.4001e-05) ├»
                           ░  ┌─────┴───┴─────┐ ├───────────────┬┘»
  q_1: ────────────────────░──┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├─»
                           ░  └┬──────────────┤ ├───────────────┤ »
  q_2: ────────────────────░───┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├─»
       ┌─────────────────┐ ░ ┌─┴──────────────┴┐└─┬────────────┬┘ »
  q_3: ┤ U3(-1.9106,π,0) ├─░─┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──»
       └─────────────────┘ ░ └─┬──────────────┬┘┌─┴────────────┴─┐»
  q_4: ────────────────────░───┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├»
                           ░   └────┬───┬─────┘ ├────────────────┤»
  q_5: ────────────────────░────────┤ X ├───────┤ Ry(2.1133e-07) ├»
                           ░        └───┘       └────────────────┘»
c34_0: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_1: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_2: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_3: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_4: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_5: ═══════════════════════════════════════════════════════════»
                                                                  »
«       ┌────────────┐                                          »
«  q_0: ┤ Rz(1.5283) ├────────────────────────────────────────■─»
«       └────────────┘                      ┌───────────────┐ │ »
«  q_1: ───────────────■──────────■─────────┤ Ry(0.0055856) ├─┼─»
«                      │          │         └───────────────┘ │ »
«  q_2: ───────────────┼──────────■───────────────────────────■─»
«                      │                                        »
«  q_3: ───────────────┼────────────────────────────────────────»
«                      │                                        »
«  q_4: ───────────────┼────────────────────────────────────────»
«       ┌────────────┐ │ ┌─────────────────┐ ┌──────────────┐   »
«  q_5: ┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├─┤ Rz(-0.23807) ├───»
«       └────────────┘   └─────────────────┘ └──────────────┘   »
«c34_0: ════════════════════════════════════════════════════════»
«                                                               »
«c34_1: ════════════════════════════════════════════════════════»
«                                                               »
«c34_2: ════════════════════════════════════════════════════════»
«                                                               »
«c34_3: ════════════════════════════════════════════════════════»
«                                                               »
«c34_4: ════════════════════════════════════════════════════════»
«                                                               »
«c34_5: ════════════════════════════════════════════════════════»
«                                                               »
«                          ┌────────────────┐  ┌──────────────┐ »
«  q_0: ─────────────────■─┤ Ry(2.6997e-05) ├──┤ Rz(0.022658) ├─»
«       ┌──────────────┐ │ └─┬────────────┬─┘  └──────────────┘ »
«  q_1: ┤ Rz(-0.21686) ├─┼───┤ U(π/2,0,π) ├─────────────────────»
«       ├─────────────┬┘ │   ├────────────┤     ┌────────────┐  »
«  q_2: ┤ Ry(0.02067) ├──┼───┤ Rz(1.2696) ├─────┤ U(π/2,0,π) ├──»
«       └─────────────┘  │   └────────────┘  ┌──┴────────────┴─┐»
«  q_3: ─────────────────┼─────────■─────────┤ Ry(-1.0324e-05) ├»
«                        │         │         └─┬──────────────┬┘»
«  q_4: ─────────────────■─────────■───────────┤ Ry(0.025251) ├─»
«                

In [50]:
len(multi_exp_circuits[0])

408

In [50]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [11,13,14,9,8,12],
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

jobset_id:  4dd95408963a4976a616907e87e0d113-1670513459790566
job_id:  1da566b0-7e4f-4efe-985f-1d440989cfc5
Job Status: job has successfully run


In [51]:
frag_datas[1]

[Result(backend_name='qasm_simulator', backend_version='0.11.1', qobj_id='ede40121-8d4f-40a5-a4b5-11a7ab537028', job_id='1da566b0-7e4f-4efe-985f-1d440989cfc5', success=True, results=[ExperimentResult(shots=10000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4989, '0x1': 5011}), header=QobjExperimentHeader(clbit_labels=[['c1', 0], ['c1', 1], ['c1', 2]], creg_sizes=[['c1', 3]], global_phase=5.156311348786247e-06, memory_slots=3, metadata={}, n_qubits=3, name="(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z'))", qreg_sizes=[['q', 3]], qubit_labels=[['q', 0], ['q', 1], ['q', 2]]), status=DONE, seed_simulator=366906276, metadata={'parallel_state_update': 8, 'parallel_shots': 1, 'sample_measure_time': 0.002081825, 'noise': 'ideal', 'batched_shots_optimization': False, 'remapped_qubits': False, 'device': 'CPU', 'active_input_qubits': [0, 1, 2], 'measure_sampling': True, 'num_clbits': 3, 'input_qubit_map': [[2, 2], [1, 1], [0, 0]], 'num_qubits': 3, 'method': 'statevector', 'fusion':

In [117]:
# from qiskit.providers.ibmq.managed import IBMQJobManager
# job_manager = IBMQJobManager()

In [118]:
# job_set = job_manager.retrieve_job_set(job_set_id='973c2fdceaf547389a770d19aad463d8-1669966038684177', provider=provider)

In [119]:
# managed_results = job_set.results()

In [120]:
# frag_datas1_copy = frag_datas[1].copy()

In [52]:
frag_datas[0] = [frag_datas[0][0].combine_results()]

In [53]:
# frag_datas[0] = run_circuits(multi_exp_circuits[0], initial_layout = None,
#                              backend = simulation_backend, shots = shots // total_variants,
#                              optimization_level = 3, monitor_jobs = True, noise_model = None)

In [54]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z'))"

In [55]:
num_tomo_circuits

34

In [56]:
test = [[], []]

In [57]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [58]:
frag_datas[1][0].get_counts()

[{'000': 4989, '001': 5011},
 {'001': 2508, '100': 2513, '000': 2454, '101': 2525},
 {'001': 2538, '100': 2552, '101': 2480, '000': 2430},
 {'001': 1729, '000': 1642, '011': 3317, '010': 3312},
 {'101': 837,
  '010': 1666,
  '000': 807,
  '111': 1658,
  '011': 1636,
  '001': 828,
  '100': 847,
  '110': 1721},
 {'101': 807,
  '010': 1742,
  '000': 850,
  '100': 813,
  '001': 786,
  '110': 1623,
  '011': 1685,
  '111': 1694},
 {'001': 1667, '000': 1681, '011': 3294, '010': 3358},
 {'101': 853,
  '001': 792,
  '110': 1704,
  '100': 820,
  '000': 785,
  '010': 1706,
  '011': 1668,
  '111': 1672},
 {'101': 819,
  '110': 1669,
  '001': 825,
  '100': 849,
  '010': 1696,
  '000': 824,
  '011': 1650,
  '111': 1668},
 {'001': 1670, '011': 3352, '000': 1638, '010': 3340},
 {'010': 1661,
  '000': 890,
  '001': 778,
  '110': 1664,
  '100': 854,
  '101': 829,
  '011': 1691,
  '111': 1633},
 {'101': 842,
  '011': 1682,
  '111': 1633,
  '100': 825,
  '001': 825,
  '110': 1619,
  '010': 1735,
  '000': 

In [59]:
frag_datas[0][0].get_counts()

[{'000000': 280,
  '000001': 256,
  '010000': 295,
  '010001': 260,
  '010010': 495,
  '010011': 454,
  '010100': 274,
  '010101': 274,
  '010110': 511,
  '010111': 449,
  '011001': 1,
  '011010': 5,
  '011011': 2,
  '011100': 3,
  '011101': 1,
  '011110': 1,
  '011111': 3,
  '000010': 475,
  '100000': 228,
  '100001': 223,
  '100010': 284,
  '100011': 255,
  '100100': 230,
  '100101': 218,
  '100110': 269,
  '100111': 263,
  '101000': 3,
  '101010': 4,
  '101011': 4,
  '101100': 2,
  '101101': 5,
  '101110': 4,
  '101111': 2,
  '000011': 413,
  '110000': 241,
  '110001': 219,
  '110010': 272,
  '110011': 279,
  '110100': 260,
  '110101': 231,
  '110110': 316,
  '110111': 279,
  '111000': 5,
  '111001': 4,
  '111010': 4,
  '111011': 4,
  '111100': 3,
  '111101': 3,
  '111110': 4,
  '111111': 5,
  '000100': 258,
  '000101': 274,
  '000110': 439,
  '000111': 424,
  '001000': 4,
  '001001': 2,
  '001010': 6,
  '001011': 5,
  '001100': 4,
  '001101': 1,
  '001110': 5,
  '001111': 3},
 {'00

In [60]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0',) ('Z',) [3] [3]
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'X', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'X', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'X', 'Z', 'Z')) ('S0',) ('X',) [3] [3]
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Y', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Y', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Y', 'Z', 'Z')) ('S0',) ('Y',) [3] [3]
(('S0', 'S0', 'S0', 'S1', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S1', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S1', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S1',) ('Z',) [3] [3]
(('S0', 'S0', 'S0', 'S1', 'S0', 'S0'), ('Z', 'Z', 'Z', 'X', 'Z', 'Z'

In [61]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [62]:
len(organized_data)

34

In [63]:
organized_data[2]

[{'00000': {(('S0',), ('Zp',)): 89,
   (('S0',), ('Zm',)): 1,
   (('S0',), ('Xp',)): 68,
   (('S0',), ('Xm',)): 39,
   (('S0',), ('Yp',)): 67,
   (('S0',), ('Ym',)): 44,
   (('S1',), ('Zp',)): 43,
   (('S1',), ('Zm',)): 84,
   (('S1',), ('Xp',)): 67,
   (('S1',), ('Xm',)): 44,
   (('S1',), ('Yp',)): 94,
   (('S1',), ('Ym',)): 11,
   (('S2',), ('Zp',)): 42,
   (('S2',), ('Zm',)): 66,
   (('S2',), ('Xp',)): 82,
   (('S2',), ('Xm',)): 21,
   (('S2',), ('Yp',)): 33,
   (('S2',), ('Ym',)): 75,
   (('S3',), ('Zp',)): 33,
   (('S3',), ('Zm',)): 84,
   (('S3',), ('Xp',)): 13,
   (('S3',), ('Xm',)): 100,
   (('S3',), ('Yp',)): 39,
   (('S3',), ('Ym',)): 83},
  '00001': {(('S0',), ('Zp',)): 114,
   (('S0',), ('Xp',)): 52,
   (('S0',), ('Xm',)): 37,
   (('S0',), ('Yp',)): 40,
   (('S0',), ('Ym',)): 39,
   (('S1',), ('Zp',)): 34,
   (('S1',), ('Zm',)): 64,
   (('S1',), ('Xp',)): 51,
   (('S1',), ('Xm',)): 44,
   (('S1',), ('Yp',)): 90,
   (('S1',), ('Ym',)): 10,
   (('S2',), ('Zp',)): 25,
   (('S2

In [64]:
organized_data[-1][-1]

{'01': {(('S0',), ('Zp',)): 5073,
  (('S0',), ('Xp',)): 2541,
  (('S0',), ('Xm',)): 2463,
  (('S0',), ('Yp',)): 2563,
  (('S0',), ('Ym',)): 2437,
  (('S1',), ('Zp',)): 1620,
  (('S1',), ('Xp',)): 868,
  (('S1',), ('Xm',)): 865,
  (('S1',), ('Yp',)): 799,
  (('S1',), ('Ym',)): 857,
  (('S2',), ('Zp',)): 1618,
  (('S2',), ('Xp',)): 837,
  (('S2',), ('Xm',)): 812,
  (('S2',), ('Ym',)): 862,
  (('S2',), ('Yp',)): 805,
  (('S3',), ('Zp',)): 1704,
  (('S3',), ('Xp',)): 861,
  (('S3',), ('Xm',)): 870,
  (('S3',), ('Ym',)): 759,
  (('S3',), ('Yp',)): 853,
  (('S0',), ('Zm',)): 0,
  (('S1',), ('Zm',)): 0,
  (('S2',), ('Zm',)): 0,
  (('S3',), ('Zm',)): 0},
 '00': {(('S0',), ('Zp',)): 4927,
  (('S0',), ('Xp',)): 2543,
  (('S0',), ('Xm',)): 2453,
  (('S0',), ('Ym',)): 2502,
  (('S0',), ('Yp',)): 2498,
  (('S1',), ('Zp',)): 1685,
  (('S1',), ('Xm',)): 835,
  (('S1',), ('Xp',)): 826,
  (('S1',), ('Ym',)): 816,
  (('S1',), ('Yp',)): 841,
  (('S2',), ('Zp',)): 1688,
  (('S2',), ('Xm',)): 800,
  (('S2'

In [72]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'00000': {(('S0',), ('Zp',)): 280, (('S0',), ('Zm',)): 4, (('S0',), ('Xp',)): 150, (('S0',), ('Xm',)): 117, (('S0',), ('Yp',)): 137, (('S0',), ('Ym',)): 125, (('S1',), ('Zp',)): 99, (('S1',), ('Zm',)): 195, (('S1',), ('Xp',)): 137, (('S1',), ('Xm',)): 136, (('S1',), ('Yp',)): 271, (('S1',), ('Ym',)): 20, (('S2',), ('Zp',)): 103, (('S2',), ('Zm',)): 198, (('S2',), ('Xp',)): 262, (('S2',), ('Xm',)): 39, (('S2',), ('Yp',)): 75, (('S2',), ('Ym',)): 204, (('S3',), ('Zp',)): 79, (('S3',), ('Zm',)): 185, (('S3',), ('Xp',)): 33, (('S3',), ('Xm',)): 256, (('S3',), ('Yp',)): 90, (('S3',), ('Ym',)): 178}, '00001': {(('S0',), ('Zp',)): 256, (('S0',), ('Zm',)): 2, (('S0',), ('Xp',)): 133, (('S0',), ('Xm',)): 108, (('S0',), ('Yp',)): 136, (('S0',), ('Ym',)): 122, (('S1',), ('Zp',)): 95, (('S1',), ('Zm',)): 168, (('S1',), ('Xp',)): 156, (('S1',), ('Xm',)): 135, (('S1',), ('Yp',)): 232, (('S1',), ('Ym',)): 11, (('S2',), ('Zp',)): 92, (('S2',), ('Zm',)): 188, (('S2',), ('Xp',)): 216, (('S2',), ('Xm',)

{'01': {(('S0',), ('Zp',)): 4984, (('S0',), ('Xp',)): 2452, (('S0',), ('Xm',)): 2516, (('S0',), ('Ym',)): 2568, (('S0',), ('Yp',)): 2504, (('S1',), ('Zp',)): 1651, (('S1',), ('Xm',)): 804, (('S1',), ('Xp',)): 860, (('S1',), ('Ym',)): 825, (('S1',), ('Yp',)): 779, (('S2',), ('Zp',)): 1667, (('S2',), ('Xm',)): 784, (('S2',), ('Xp',)): 859, (('S2',), ('Ym',)): 805, (('S2',), ('Yp',)): 858, (('S3',), ('Zp',)): 1697, (('S3',), ('Xp',)): 848, (('S3',), ('Xm',)): 922, (('S3',), ('Ym',)): 800, (('S3',), ('Yp',)): 892, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '00': {(('S0',), ('Zp',)): 5016, (('S0',), ('Xm',)): 2496, (('S0',), ('Xp',)): 2536, (('S0',), ('Ym',)): 2435, (('S0',), ('Yp',)): 2493, (('S1',), ('Zp',)): 1635, (('S1',), ('Xm',)): 911, (('S1',), ('Xp',)): 830, (('S1',), ('Yp',)): 840, (('S1',), ('Ym',)): 877, (('S2',), ('Zp',)): 1646, (('S2',), ('Xm',)): 832, (('S2',), ('Xp',)): 823, (('S2',), ('Ym',)): 849, (('S2',), ('Yp',)): 787, ((

{'00000': {(('S0',), ('Zp',)): 567, (('S0',), ('Zm',)): 12, (('S0',), ('Xp',)): 294, (('S0',), ('Xm',)): 282, (('S0',), ('Yp',)): 307, (('S0',), ('Ym',)): 319, (('S1',), ('Zp',)): 180, (('S1',), ('Zm',)): 395, (('S1',), ('Xp',)): 321, (('S1',), ('Xm',)): 324, (('S1',), ('Yp',)): 578, (('S1',), ('Ym',)): 30, (('S2',), ('Zp',)): 191, (('S2',), ('Zm',)): 379, (('S2',), ('Xp',)): 552, (('S2',), ('Xm',)): 94, (('S2',), ('Yp',)): 142, (('S2',), ('Ym',)): 519, (('S3',), ('Zp',)): 196, (('S3',), ('Zm',)): 377, (('S3',), ('Xp',)): 62, (('S3',), ('Xm',)): 561, (('S3',), ('Yp',)): 205, (('S3',), ('Ym',)): 427}, '00001': {(('S0',), ('Zp',)): 655, (('S0',), ('Zm',)): 11, (('S0',), ('Xp',)): 297, (('S0',), ('Xm',)): 294, (('S0',), ('Yp',)): 246, (('S0',), ('Ym',)): 291, (('S1',), ('Zp',)): 191, (('S1',), ('Zm',)): 400, (('S1',), ('Xp',)): 308, (('S1',), ('Xm',)): 250, (('S1',), ('Yp',)): 509, (('S1',), ('Ym',)): 50, (('S2',), ('Zp',)): 195, (('S2',), ('Zm',)): 416, (('S2',), ('Xp',)): 469, (('S2',),

{'00': {(('S0',), ('Zp',)): 5085, (('S0',), ('Xm',)): 2459, (('S0',), ('Xp',)): 2466, (('S0',), ('Ym',)): 2428, (('S0',), ('Yp',)): 2490, (('S1',), ('Zp',)): 1673, (('S1',), ('Xm',)): 838, (('S1',), ('Xp',)): 850, (('S1',), ('Yp',)): 808, (('S1',), ('Ym',)): 874, (('S2',), ('Zp',)): 1612, (('S2',), ('Xp',)): 819, (('S2',), ('Xm',)): 882, (('S2',), ('Ym',)): 825, (('S2',), ('Yp',)): 872, (('S3',), ('Zp',)): 1648, (('S3',), ('Xm',)): 786, (('S3',), ('Xp',)): 853, (('S3',), ('Ym',)): 795, (('S3',), ('Yp',)): 807, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '01': {(('S0',), ('Zp',)): 4915, (('S0',), ('Xp',)): 2574, (('S0',), ('Xm',)): 2501, (('S0',), ('Yp',)): 2526, (('S0',), ('Ym',)): 2556, (('S1',), ('Zp',)): 1657, (('S1',), ('Xp',)): 790, (('S1',), ('Xm',)): 869, (('S1',), ('Ym',)): 872, (('S1',), ('Yp',)): 801, (('S2',), ('Zp',)): 1644, (('S2',), ('Xm',)): 822, (('S2',), ('Xp',)): 885, (('S2',), ('Ym',)): 851, (('S2',), ('Yp',)): 853, ((

{'01': {(('S0',), ('Zp',)): 4948, (('S0',), ('Xp',)): 2459, (('S0',), ('Xm',)): 2517, (('S0',), ('Yp',)): 2532, (('S0',), ('Ym',)): 2435, (('S1',), ('Zp',)): 1645, (('S1',), ('Xm',)): 812, (('S1',), ('Xp',)): 824, (('S1',), ('Yp',)): 842, (('S1',), ('Ym',)): 831, (('S2',), ('Zp',)): 1707, (('S2',), ('Xm',)): 824, (('S2',), ('Xp',)): 810, (('S2',), ('Ym',)): 856, (('S2',), ('Yp',)): 838, (('S3',), ('Zp',)): 1691, (('S3',), ('Xp',)): 839, (('S3',), ('Xm',)): 838, (('S3',), ('Yp',)): 845, (('S3',), ('Ym',)): 869, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '00': {(('S0',), ('Zp',)): 5052, (('S0',), ('Xm',)): 2580, (('S0',), ('Xp',)): 2444, (('S0',), ('Ym',)): 2520, (('S0',), ('Yp',)): 2513, (('S1',), ('Zp',)): 1697, (('S1',), ('Xm',)): 861, (('S1',), ('Xp',)): 861, (('S1',), ('Yp',)): 850, (('S1',), ('Ym',)): 831, (('S2',), ('Zp',)): 1720, (('S2',), ('Xp',)): 849, (('S2',), ('Xm',)): 789, (('S2',), ('Ym',)): 856, (('S2',), ('Yp',)): 831, ((

...

{'00': {(('S0',), ('Zp',)): 10000, (('S0',), ('Xp',)): 4980, (('S0',), ('Xm',)): 5020, (('S0',), ('Yp',)): 4994, (('S0',), ('Ym',)): 5006, (('S1',), ('Zp',)): 3377, (('S1',), ('Xm',)): 1710, (('S1',), ('Xp',)): 1607, (('S1',), ('Ym',)): 1640, (('S1',), ('Yp',)): 1665, (('S2',), ('Zp',)): 3386, (('S2',), ('Xp',)): 1642, (('S2',), ('Xm',)): 1628, (('S2',), ('Ym',)): 1713, (('S2',), ('Yp',)): 1696, (('S3',), ('Zp',)): 3285, (('S3',), ('Xm',)): 1643, (('S3',), ('Xp',)): 1700, (('S3',), ('Yp',)): 1663, (('S3',), ('Ym',)): 1667, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '11': {(('S1',), ('Zp',)): 6623, (('S1',), ('Xm',)): 3349, (('S1',), ('Xp',)): 3334, (('S1',), ('Ym',)): 3398, (('S1',), ('Yp',)): 3297, (('S2',), ('Zp',)): 6614, (('S2',), ('Xp',)): 3369, (('S2',), ('Xm',)): 3361, (('S2',), ('Ym',)): 3314, (('S2',), ('Yp',)): 3277, (('S3',), ('Zp',)): 6715, (('S3',), ('Xp',)): 3323, (('S3',), ('Xm',)): 3334, (('S3',), ('Yp',)): 3305, (('S3',

{'00': {(('S0',), ('Zp',)): 10000, (('S0',), ('Xp',)): 5031, (('S0',), ('Xm',)): 4969, (('S0',), ('Ym',)): 5121, (('S0',), ('Yp',)): 4879, (('S1',), ('Zp',)): 3379, (('S1',), ('Xm',)): 1604, (('S1',), ('Xp',)): 1662, (('S1',), ('Ym',)): 1596, (('S1',), ('Yp',)): 1659, (('S2',), ('Zp',)): 3401, (('S2',), ('Xm',)): 1632, (('S2',), ('Xp',)): 1675, (('S2',), ('Ym',)): 1678, (('S2',), ('Yp',)): 1693, (('S3',), ('Zp',)): 3247, (('S3',), ('Xm',)): 1606, (('S3',), ('Xp',)): 1681, (('S3',), ('Yp',)): 1651, (('S3',), ('Ym',)): 1637, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '11': {(('S1',), ('Zp',)): 6621, (('S1',), ('Xp',)): 3398, (('S1',), ('Xm',)): 3336, (('S1',), ('Ym',)): 3369, (('S1',), ('Yp',)): 3376, (('S2',), ('Zp',)): 6599, (('S2',), ('Xm',)): 3274, (('S2',), ('Xp',)): 3419, (('S2',), ('Ym',)): 3313, (('S2',), ('Yp',)): 3316, (('S3',), ('Zp',)): 6753, (('S3',), ('Xp',)): 3380, (('S3',), ('Xm',)): 3333, (('S3',), ('Yp',)): 3399, (('S3',

{'00': {(('S0',), ('Zp',)): 4912, (('S0',), ('Xm',)): 2500, (('S0',), ('Xp',)): 2424, (('S0',), ('Yp',)): 2470, (('S0',), ('Ym',)): 2491, (('S1',), ('Zp',)): 1689, (('S1',), ('Xm',)): 796, (('S1',), ('Xp',)): 871, (('S1',), ('Ym',)): 870, (('S1',), ('Yp',)): 838, (('S2',), ('Zp',)): 1686, (('S2',), ('Xp',)): 850, (('S2',), ('Xm',)): 812, (('S2',), ('Ym',)): 866, (('S2',), ('Yp',)): 785, (('S3',), ('Zp',)): 1692, (('S3',), ('Xm',)): 822, (('S3',), ('Xp',)): 821, (('S3',), ('Ym',)): 822, (('S3',), ('Yp',)): 815, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '01': {(('S0',), ('Zp',)): 5088, (('S0',), ('Xp',)): 2584, (('S0',), ('Xm',)): 2492, (('S0',), ('Yp',)): 2555, (('S0',), ('Ym',)): 2484, (('S1',), ('Zp',)): 1713, (('S1',), ('Xm',)): 869, (('S1',), ('Xp',)): 847, (('S1',), ('Ym',)): 851, (('S1',), ('Yp',)): 848, (('S2',), ('Zp',)): 1603, (('S2',), ('Xm',)): 802, (('S2',), ('Xp',)): 816, (('S2',), ('Ym',)): 837, (('S2',), ('Yp',)): 809, ((

{'00': {(('S0',), ('Zp',)): 4933, (('S0',), ('Xp',)): 2476, (('S0',), ('Xm',)): 2488, (('S0',), ('Ym',)): 2534, (('S0',), ('Yp',)): 2483, (('S1',), ('Zp',)): 1658, (('S1',), ('Xm',)): 816, (('S1',), ('Xp',)): 818, (('S1',), ('Yp',)): 795, (('S1',), ('Ym',)): 842, (('S2',), ('Zp',)): 1670, (('S2',), ('Xp',)): 820, (('S2',), ('Xm',)): 841, (('S2',), ('Ym',)): 801, (('S2',), ('Yp',)): 827, (('S3',), ('Zp',)): 1667, (('S3',), ('Xm',)): 771, (('S3',), ('Xp',)): 872, (('S3',), ('Ym',)): 870, (('S3',), ('Yp',)): 865, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '01': {(('S0',), ('Zp',)): 5067, (('S0',), ('Xp',)): 2526, (('S0',), ('Xm',)): 2510, (('S0',), ('Ym',)): 2540, (('S0',), ('Yp',)): 2443, (('S1',), ('Zp',)): 1722, (('S1',), ('Xm',)): 882, (('S1',), ('Xp',)): 810, (('S1',), ('Ym',)): 789, (('S1',), ('Yp',)): 841, (('S2',), ('Zp',)): 1685, (('S2',), ('Xp',)): 798, (('S2',), ('Xm',)): 837, (('S2',), ('Ym',)): 769, (('S2',), ('Yp',)): 833, ((

{'01': {(('S0',), ('Zp',)): 5073, (('S0',), ('Xp',)): 2541, (('S0',), ('Xm',)): 2463, (('S0',), ('Yp',)): 2563, (('S0',), ('Ym',)): 2437, (('S1',), ('Zp',)): 1620, (('S1',), ('Xp',)): 868, (('S1',), ('Xm',)): 865, (('S1',), ('Yp',)): 799, (('S1',), ('Ym',)): 857, (('S2',), ('Zp',)): 1618, (('S2',), ('Xp',)): 837, (('S2',), ('Xm',)): 812, (('S2',), ('Ym',)): 862, (('S2',), ('Yp',)): 805, (('S3',), ('Zp',)): 1704, (('S3',), ('Xp',)): 861, (('S3',), ('Xm',)): 870, (('S3',), ('Ym',)): 759, (('S3',), ('Yp',)): 853, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '00': {(('S0',), ('Zp',)): 4927, (('S0',), ('Xp',)): 2543, (('S0',), ('Xm',)): 2453, (('S0',), ('Ym',)): 2502, (('S0',), ('Yp',)): 2498, (('S1',), ('Zp',)): 1685, (('S1',), ('Xm',)): 835, (('S1',), ('Xp',)): 826, (('S1',), ('Ym',)): 816, (('S1',), ('Yp',)): 841, (('S2',), ('Zp',)): 1688, (('S2',), ('Xm',)): 800, (('S2',), ('Xp',)): 804, (('S2',), ('Ym',)): 784, (('S2',), ('Yp',)): 830, ((

In [73]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [74]:
from utils.utils import filter_results

In [75]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [76]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [77]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [78]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_counts(norm_filtered_lists[i]), Pauli = Pauli_tuple[0])  

In [79]:
final_expect_val

(-3.3103598298269445+0j)

In [80]:
import json
with open('BeH2_auckland_check5.json', 'w') as f:
    json.dump(norm_filtered_lists, f)

In [81]:
mit = mthree.M3Mitigation(device)
mit.cals_from_file(cals_file = 'mit_data_12_13_14_11_8_9')

In [82]:
def update_dict(quasis, total_shots = None):
    for key in quasis.keys():
        quasis[key] = quasis[key]*total_shots

In [83]:
mit_lists = []
for i in range(0, len(pauli_commute)):
    total_shots = total_counts(norm_filtered_lists[i])
    quasis = mit.apply_correction(norm_filtered_lists[i], [12,13,14,11,8,9])
    update_dict(quasis, total_shots = total_shots)
    mit_lists.append(quasis)

In [84]:
miti_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        miti_expect_val += coeff * evaluation(mit_lists[i], shots = total_counts(mit_lists[i]), Pauli = Pauli_tuple[0]) 
miti_expect_val

(-3.3499784566899766+0j)

In [85]:
with open('BeH2_auckland_check5_mit.json', 'w') as f:
    json.dump(mit_lists, f)